In [1]:
# Initialization
%cd ../covid_households/
import recipes
import utilities
import traits

import tqdm
from multiprocessing import Pool

/Users/thayer/covid_households/covid_households


# What is this notebook for?

We conduct our simulations over a range of values for each of our three different parameters, $s_{80}$, $p_{80}$, and $\text{SAR}$. But these parameters are not direct inputs into our model, they are complex expressions of properties of the distributions of relative susceptibility and infectivity ($s_{80}$ and $p_{80}$) in the population or the average risk of infection from a household contact ($\text{SAR}$). For full information about these parameters, see the Methods and Supplemental Methods sections.

To convert these parameters to actual model parameters (the mean & variance of distributions; or $\beta$, the probability/time of infection) we use numerical methods. For the overwhelming majority of parameter combinations, this works great. But when $p_{80}$ or $s_{80}$ is small and $\text{SAR}$ is high, we cannot solve for a $\beta$ that actually produces the desired $\text{SAR}$. There is so much heterogeneity (and thus so many people that are neglibly infectious or susceptible) that we can't solve for an appropriately high $\beta$ given that $\beta < 1$.

We want to drop these points of our 3d grid in parameter space so that the likelihood surface does not include points with an unrealistic $\beta$. To do that, we first have to find every point where the residual from the numerical fit is higher than our tolerance ($10^{-5}$).

We define the region over which we simulate by enumerating each of its axes. In order to compute in parallel, we also make a `coordinate_stream` generator that yields coordinate pairs for the entire region in sequence.

In [ ]:
import numpy as np
s80_axis = np.linspace(0.02, 0.80, 40)
p80_axis = np.linspace(0.02, 0.80, 40)
sar_axis = np.linspace(0.01, 0.60, 60)

def coordinate_stream(axis1, axis2, axis3):
    for v1 in axis1:
        for v2 in axis2:
            for v3 in axis3:
                yield (v1, v2, v3)

Using Python's multiprocessing functionality, we iterate over each point in the region and apply the `calculate_residual` function from `utilities` in order to find the difference between the expected $\text{SAR}$ and the $\text{SAR}$ that is actually implied by $\beta$ and the traits.

In [ ]:
with Pool(4) as p:
    total = len(s80_axis) * len(p80_axis) * len(sar_axis)
    residuals = list(tqdm.tqdm(
        p.imap(utilities.residual_wrapper, coordinate_stream(s80_axis, p80_axis, sar_axis)),
        total=total
    ))

A "crib" is a cheat sheet that we use to precalculate the mappings between our parameters and the model parameters we need to simulate forwards in time.

We want to mirror its structure but find the cells were the residual is greater than our tolerance.

In [ ]:
beta_crib_copy = model_inputs.S80_P80_SAR_Inputs.beta_crib.copy()
beta_crib_copy['residuals'] = list(residuals)

In [ ]:
beta_crib_copy['bad beta'] = beta_crib_copy['residuals'] > 10e-5

In [ ]:
beta_crib_copy[beta_crib_copy['bad beta']]

In [ ]:
beta_crib_copy.to_csv('./problematic_parameter_combinations.csv')

In [ ]:
((beta_crib_copy[beta_crib_copy['bad beta']].reset_index()['SAR'] < 0.35) & (beta_crib_copy[beta_crib_copy['bad beta']].reset_index()['p80'] > 0.10) &(beta_crib_copy[beta_crib_copy['bad beta']].reset_index()['s80'] > 0.10)).sum()

For each bad beta, now that we've made a correction to our recipe for solving for the beta, we want to recalculate the beta.

In [2]:
import pandas as pd
beta_crib_copy = pd.read_csv('../data/problematic_parameter_combinations.csv').set_index(['s80','p80','SAR'])

In [50]:
bad_view = beta_crib_copy[beta_crib_copy['bad beta']]
bad_indices = bad_view.index
bad_indices

MultiIndex([(0.02, 0.02, 0.08),
            (0.02, 0.02, 0.09),
            (0.02, 0.02,  0.1),
            (0.02, 0.02, 0.11),
            (0.02, 0.02, 0.12),
            (0.02, 0.02, 0.13),
            (0.02, 0.02, 0.14),
            (0.02, 0.02, 0.15),
            (0.02, 0.02, 0.16),
            (0.02, 0.02, 0.17),
            ...
            ( 0.8, 0.12, 0.56),
            ( 0.8, 0.12, 0.57),
            ( 0.8, 0.12, 0.58),
            ( 0.8, 0.12, 0.59),
            ( 0.8, 0.12,  0.6),
            ( 0.8, 0.14, 0.57),
            ( 0.8, 0.14, 0.58),
            ( 0.8, 0.14, 0.59),
            ( 0.8, 0.14,  0.6),
            ( 0.8, 0.16,  0.6)],
           names=['s80', 'p80', 'SAR'], length=14389)

In [78]:
b_frame = bad_indices.to_frame()
b_frame = b_frame[(b_frame['s80'] != 0.02) & (b_frame['p80'] != 0.02)]
b_frame.reset_index(drop=True).to_csv('../data/indices_to_rerun.csv')

In [81]:
new_frame_test = pd.read_csv('../data/indices_to_rerun.csv')
new_frame_test.set_index(['s80', 'p80', 'SAR']).index

MultiIndex([(0.04, 0.04, 0.13),
            (0.04, 0.04, 0.14),
            (0.04, 0.04, 0.15),
            (0.04, 0.04, 0.16),
            (0.04, 0.04, 0.17),
            (0.04, 0.04, 0.18),
            (0.04, 0.04, 0.19),
            (0.04, 0.04,  0.2),
            (0.04, 0.04, 0.21),
            (0.04, 0.04, 0.22),
            ...
            ( 0.8, 0.12, 0.56),
            ( 0.8, 0.12, 0.57),
            ( 0.8, 0.12, 0.58),
            ( 0.8, 0.12, 0.59),
            ( 0.8, 0.12,  0.6),
            ( 0.8, 0.14, 0.57),
            ( 0.8, 0.14, 0.58),
            ( 0.8, 0.14, 0.59),
            ( 0.8, 0.14,  0.6),
            ( 0.8, 0.16,  0.6)],
           names=['s80', 'p80', 'SAR'], length=11320)

In [57]:
list(bad_indices.names)

['s80', 'p80', 'SAR']

In [10]:
bad_index_frame = bad_indices.to_frame().reset_index(drop=True)

In [55]:
((bad_index_frame['s80'] > 0.02) & (bad_index_frame['p80'] > 0.02) & (bad_index_frame['SAR'] <= 0.60)).sum()

11320

In [41]:
import numpy as np
np.linspace(0.36, 0.8, 23)

array([0.36, 0.38, 0.4 , 0.42, 0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56,
       0.58, 0.6 , 0.62, 0.64, 0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78,
       0.8 ])

In [ ]:
new_betas = []
for i in bad_indices:
    old_beta = bad_view.loc[i]['beta']
    new_beta = model_inputs.S80_P80_SAR_Inputs(*i).to_normal_inputs()['household_beta']
    print(old_beta, beta)
    new_betas.append(new_beta)

In [ ]:
with Pool(4) as p:
    total = len(bad_indices)
    residuals = list(tqdm.tqdm(
        p.imap(model_inputs.S80_P80_SAR_Inputs.wrapped_beta_from_point, (i for i in bad_indices)),
        total=total
    ))

In [ ]:
len(residuals)

In [ ]:
new_betas = residuals.copy()

In [ ]:
bad_view

In [ ]:
new_betas[0]

In [ ]:
import numpy as np
import scipy.integrate
def SAR(beta, sus, inf, tau):
    generalized_period = traits.LognormalTrait(tau.mu + inf.mu + sus.mu, np.sqrt(tau.sigma**2 + inf.sigma**2 + sus.sigma**2)).distribution
    integral, abserror = scipy.integrate.quad(lambda x: generalized_period.pdf(x) * np.exp(-1 * beta * x), 0, np.inf)
    sar = 1 - integral
    return sar

In [ ]:
sus = traits.LognormalTrait(model_inputs.S80_P80_SAR_Inputs.s80_crib.loc[0.02].mu, model_inputs.S80_P80_SAR_Inputs.s80_crib.loc[0.02].sigma)
inf = traits.LognormalTrait(model_inputs.S80_P80_SAR_Inputs.p80_crib.loc[0.02].mu, model_inputs.S80_P80_SAR_Inputs.p80_crib.loc[0.02].sigma)

sus, inf

In [ ]:
sus.distribution.var()

In [ ]:
other_solves = []
for s80 in model_inputs.S80_P80_SAR_Inputs.s80_crib.index:
    trait = traits.LognormalTrait.from_natural_mean_variance(1.0, utilities.lognormal_s80_solve(s80).x[0])
    other_solves.append((trait.mu, trait.sigma, trait.distribution.var()))

In [ ]:
model_inputs.S80_P80_SAR_Inputs.wrapped_beta_from_point((0.02, 0.02, 0.08))

In [ ]:
model_inputs.S80_P80_SAR_Inputs(0.02, 0.02, 0.08).to_normal_inputs(use_beta_crib=False)

In [ ]:
g_tau = utilities.lognormal_calculate_generalized_period(sus, inf)
obj = utilities.lognormal_SAR_objective_function_crafter(0.08, g_tau)

In [ ]:
obj(0.999)

In [ ]:
utilities.beta_from_sar_and_lognormal_traits(0.08, sus, inf)

In [ ]:
model_inputs.S80_P80_SAR_Inputs.wrapped_beta_from_point((0.02, 0.02, 0.08))

In [ ]:
x = model_inputs.S80_P80_SAR_Inputs(*(0.02, 0.02, 0.08))
x

In [ ]:
dir = '/Users/thayer/covid_households/new_parameters/gillespie-s80-p80-SAR/sizes-2-5/experiment-07-02-16-34'
results = recipes.Results.load(dir)
results.df